In [1]:
library(tidyverse)
library(WGCNA)
library(cowplot)
library(matrixStats)

# Custom package
library(rutils)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: dynamicTreeCut
Loading required package: fastcluster

Attaching package: ‘fastcluster’

The following object is masked from ‘package:stats’:

    hclust



Attaching package: ‘WGCNA’

The following object is masked from ‘package:stats’:

    cor


********************************************************
Note: As of version 1.0.0, cowplot does not change the
  default ggplot2 theme anymore. To recover the previous
  behavior, execute:
  theme_set(theme_cowplot())
********************************************************


Attaching package: ‘matrixStats’

The following object is masked 

In [2]:
options(stringsAsFactors = FALSE)
enableWGCNAThreads()

Allowing parallel execution with up to 15 working processes.


In [3]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-UCS", "TCGA-UCEC", "TCGA-OV")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")

event_code <- list("Alive" = 0, "Dead" = 1)

In [4]:
dset_idx <- 1

# Load data

In [5]:
lnames <- load(file = paste0(dirs$data_dir, "/saved_network_objects/", unified_dsets[dset_idx], "_tumor_data.RData"))
lnames <- load(file = paste0(dirs$data_dir, "/saved_network_objects/", unified_dsets[dset_idx], "_tumor_network.RData"))

In [6]:
n_genes <- ncol(data_expr)
n_samples <- nrow(data_expr)

In [7]:
mes_0 <- moduleEigengenes(data_expr, module_colors)$eigengenes
mes <- orderMEs(mes_0)

# Module significance for survival analysis

In [8]:
module_coxph_res <- standardScreeningCensoredTime(
    datExpr = mes,
    time = data_traits$survival_time, 
    event = data_traits$vital_status, 
    fastCalculation = FALSE,
    qValues = TRUE
)

In [9]:
module_coxph_res

ID,pvalueWald,qvalueWald,pvalueLogrank,qvalueLogrank,pvalueDeviance,qvalueDeviance,corDeviance,HazardRatio,CI.LowerLimitHR,CI.UpperLimitHR,C.index
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MEblue,0.71172530,0.12347540,0.71165627,0.12346342,0.82953823,0.09627574,0.01356141,2.15624060,0.0366271547,126.937884,0.4892626
MEgreen,0.01240274,0.02151720,0.01203731,0.02088322,0.01069808,0.01365775,0.15941527,103.37668283,2.7252999845,3921.307237,0.3983638
MEpurple,0.45960348,0.08859492,0.45866092,0.08841323,0.50917033,0.07222603,0.04156034,3.95462325,0.1033984418,151.250297,0.4396091
MEmagenta,0.07464291,0.02589921,0.06927315,0.02461806,0.11258111,0.02874542,0.09962103,23.58744985,0.7301943972,761.944754,0.4950574
MEpink,0.44983638,0.08859492,0.44909764,0.08841323,0.46876440,0.07222603,0.04560698,3.92842831,0.1129835192,136.591152,0.4552892
MEred,0.04860866,0.02589921,0.04867081,0.02461806,0.08742551,0.02790303,0.10725144,62.77720324,1.0255278657,3842.876803,0.4382457
MEblack,0.84376026,0.13307437,0.84360954,0.13305060,0.67907814,0.08669477,0.02605610,1.47212758,0.0314590086,68.888364,0.5078968
MEyellow,0.06953267,0.02589921,0.06923378,0.02461806,0.02508277,0.01601101,-0.14018536,0.03187312,0.0007713250,1.317079,0.5965231
MEbrown,0.19783366,0.05720276,0.19758463,0.05713075,0.15583750,0.03315842,-0.08916446,0.06212839,0.0009049679,4.265274,0.5743665


In [10]:
gene_coxph_res <- standardScreeningCensoredTime(
    datExpr = data_expr,
    time = data_traits$survival_time,
    event = data_traits$vital_status,
    fastCalculation = FALSE,
    qValues = TRUE
)

Warning message in fitter(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”Warning message in fitter(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”Warning message in fitter(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”Warning message in fitter(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”Warning message in fitter(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”

In [11]:
nrow(gene_coxph_res[gene_coxph_res$qvalueLogrank < 0.05, ])
head(gene_coxph_res[gene_coxph_res$qvalueLogrank < 0.05, ])

[1] 48

,ID,pvalueWald,qvalueWald,pvalueLogrank,qvalueLogrank,pvalueDeviance,qvalueDeviance,corDeviance,HazardRatio,CI.LowerLimitHR,CI.UpperLimitHR,C.index
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
14,ANGPT2,0.0004260227,0.01477003,0.0004152209,0.01295850,0.0001682795,0.009779259,0.2326695,1.501484,1.197639,1.882416,0.3325758
50,SERPINA5,0.0029814799,0.04532241,0.0025140092,0.03617566,0.0103938028,0.090000239,0.1600313,1.244038,1.077066,1.436895,0.3789342
53,CSF2,0.0024193091,0.03958573,0.0023151464,0.03569367,0.0034924543,0.053594727,0.1819385,1.361322,1.115319,1.661584,0.3646698
67,P4HA3,0.0003874222,0.01477003,0.0004221192,0.01295850,0.0012626161,0.031446272,0.2003667,1.416678,1.168738,1.717216,0.3996137
74,CXCL2,0.0002679320,0.01249420,0.0002228013,0.01080201,0.0008489514,0.028917575,0.2071277,1.271886,1.117573,1.447505,0.3673446
108,PF4,0.0038271383,0.05570189,0.0033279970,0.04585082,0.0121688436,0.095348793,0.1566391,1.538407,1.148921,2.059930,0.3855965


# Module significance for FIGO stage

In [12]:
module_figo_cor <- WGCNA::cor(mes, data_traits[-c(1:2)])
module_figo_cor

,figo_stage_1,figo_stage_2,figo_stage_3,figo_stage_4
MEblue,-0.130419646,0.020075610,0.17758875,-0.0306603904
MEgreen,-0.069146586,0.039862877,0.07559666,-0.0380222755
MEpurple,0.034966554,0.011484307,-0.05967674,-0.0024735207
MEmagenta,0.034817860,0.001740843,-0.09945942,0.0687326277
MEpink,0.189387367,-0.084598692,-0.14027049,-0.0282833095
MEred,0.169077826,-0.074497461,-0.08466824,-0.0830931269
MEblack,-0.102602181,0.098098551,-0.06480633,0.1254048773
MEyellow,-0.004098473,0.050341150,-0.01586284,-0.0520231763
MEbrown,0.063001885,-0.038066250,-0.02707073,-0.0204201646
MEturquoise,-0.051069808,0.017751013,0.03236273,0.0234145339


In [13]:
module_figo_pvals <- corPvalueStudent(module_figo_cor, n_samples)
module_figo_pvals

,figo_stage_1,figo_stage_2,figo_stage_3,figo_stage_4
MEblue,0.037406435,0.7496970,0.004447037,0.62603420
MEgreen,0.271299563,0.5262873,0.228986730,0.54557600
MEpurple,0.578349099,0.8551945,0.342556859,0.96864725
MEmagenta,0.579967054,0.9779313,0.113110156,0.27418606
MEpink,0.002389876,0.1780733,0.025089542,0.65305715
MEred,0.006805758,0.2358466,0.177715597,0.18594841
MEblack,0.102113494,0.1181507,0.302599997,0.04543322
MEyellow,0.948073604,0.4234548,0.800978726,0.40811105
MEbrown,0.316289270,0.5451111,0.667021186,0.74554767
MEturquoise,0.416767076,0.7778741,0.606981393,0.70980786


# Find genes with high module membership (MM) and gene significance (GS)

In [14]:
gene_module_map_df <- tibble(geneID = colnames(data_expr), module = module_colors)

In [15]:
gene_mm <- WGCNA::cor(data_expr, mes, use = "p")
gene_mm_pval <- corPvalueStudent(gene_mm, nSamples = n_samples)

gene_gs <- WGCNA::cor(data_expr, data_traits, use = "p")
gene_gs_pval <- corPvalueStudent(gene_gs, nSamples = n_samples)

In [38]:
gene_mm_0 <- WGCNA::cor(data_expr, mes_0, use = "p")

In [16]:
gene_mm_df <- gene_mm %>%
    as_tibble(rownames = "geneID") %>%
    dplyr::rename_if(startsWith(colnames(.), "ME"), ~ gsub("ME", "", .)) %>%
    dplyr::rename_if(!startsWith(colnames(.), "geneID"), ~ gsub("$", "_cor", .))
gene_mm_pval_df <- gene_mm_pval %>%
    as_tibble(rownames = "geneID") %>%
    dplyr::rename_if(startsWith(colnames(.), "ME"), ~ gsub("ME", "", .)) %>%
    dplyr::rename_if(!startsWith(colnames(.), "geneID"), ~ gsub("$", "_pval", .))

In [17]:
gene_gs_df <- gene_gs %>% 
    as_tibble(rownames = "geneID") %>%
    dplyr::rename_if(!startsWith(colnames(.), "geneID"), ~ gsub("$", "_cor", .))
gene_gs_pval_df <- gene_gs_pval %>%
    as_tibble(rownames = "geneID") %>%
    dplyr::rename_if(!startsWith(colnames(.), "geneID"), ~ gsub("$", "_pval", .))

In [19]:
colnames(gene_gs_df)[startsWith(colnames(gene_gs_df), "blue")]

character(0)

In [48]:
mes

,MEblue,MEgreen,MEpurple,MEmagenta,MEpink,MEred,MEblack,MEyellow,MEbrown,MEturquoise,MEgrey
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TCGA-C5-A1BF-01B-11R-A13Y-07,-0.003256702,0.073537564,0.0341830711,-0.0131508427,0.010626381,0.076211183,0.0236326101,0.005591921,0.030573462,0.129155183,-0.0408391432
TCGA-MU-A51Y-01A-11R-A26T-07,0.043969035,-0.047504170,-0.0314698674,-0.0218445354,-0.023973306,-0.005620746,-0.0060860641,0.082991029,0.110799549,0.024041363,-0.0026598050
TCGA-EK-A2RM-01A-21R-A18M-07,-0.032155831,0.044616210,-0.0307913185,-0.0129065499,-0.020022744,0.086350881,0.0240009505,-0.022432207,0.005870983,0.096964644,-0.0574064510
TCGA-Q1-A73P-01A-11R-A32P-07,-0.087552967,-0.067152253,-0.0049366333,0.0495055955,0.187615605,0.119583810,-0.0051471091,0.013082427,0.020945195,-0.025477452,-0.0517716770
TCGA-C5-A8YT-01A-11R-A37O-07,-0.129403614,-0.047925655,-0.0007305383,0.0470770746,0.034027389,-0.057191386,0.0392001546,-0.031724194,0.182165412,0.147411200,-0.0393792670
TCGA-C5-A2LZ-01A-11R-A213-07,0.058003190,-0.030624825,-0.0323186637,-0.0173489106,-0.025172427,-0.031825678,-0.0215443279,0.013620821,-0.045479206,-0.093695541,0.0617758609
TCGA-UC-A7PI-01A-11R-A42S-07,-0.107236168,-0.095558051,-0.0162119820,0.0665165490,0.146895877,0.083641949,-0.0196236258,-0.064048599,-0.028465621,-0.059999392,-0.0526778622
TCGA-VS-A9V1-01A-11R-A42T-07,-0.096191591,-0.067100951,0.0285122602,-0.0016588757,0.192821129,0.166728195,-0.0100744069,-0.050537649,-0.046350271,-0.057565138,-0.0575903659
TCGA-FU-A5XV-01A-11R-A28H-07,0.103065210,0.060812402,-0.0274597177,-0.0274295933,-0.028479613,0.025468069,-0.0207390534,-0.020395758,-0.029940526,-0.032622149,-0.0570406794


In [20]:
joined_gs_mm_df <- gene_module_map_df %>%
    inner_join(gene_mm_df, by = "geneID") %>%
    inner_join(gene_gs_df, by = "geneID") %>%
    inner_join(gene_mm_pval_df, by = "geneID") %>%
    inner_join(gene_gs_pval_df, by = "geneID")

In [49]:
dim(joined_gs_mm_df)
head(joined_gs_mm_df)

[1] 1004   36

geneID,module,blue_cor,green_cor,purple_cor,magenta_cor,pink_cor,red_cor,black_cor,yellow_cor,⋯,yellow_pval,brown_pval,turquoise_pval,grey_pval,vital_status_pval,survival_time_pval,figo_stage_1_pval,figo_stage_2_pval,figo_stage_3_pval,figo_stage_4_pval
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PGF,blue,0.38408356,0.39967124,-0.045670879,-0.156782230,-0.44561985,-0.34464419,0.005694877,0.01330499,⋯,0.832552376,3.088557e-01,2.113275e-03,0.5240736,0.1756954,0.55313481,0.0001030322,0.45672899,0.00129834,0.09702818
TIMP4,turquoise,-0.12184826,0.12149728,0.037239103,0.305735481,0.17703564,0.09404059,0.201547845,-0.18064611,⋯,0.003799274,4.647960e-02,1.759269e-04,0.8829144,0.1164686,0.46236319,0.7257478663,0.38235585,0.77650262,0.08979772
C1QTNF6,turquoise,0.03030913,0.50141509,0.115435727,-0.094700491,0.01307332,0.29924087,-0.015937207,0.03687666,⋯,0.557755024,8.024397e-01,1.229518e-12,0.8774832,0.2419066,0.02469719,0.6431873419,0.20083579,0.47524876,0.83668832
TNC,turquoise,0.10084248,0.48756547,-0.006222013,-0.150565762,-0.06596766,0.15603347,-0.033054989,0.05412887,⋯,0.389373956,5.282683e-06,1.025253e-18,0.6855194,0.2603504,0.06997259,0.1561754917,0.63066133,0.56813078,0.26261025
PRL,purple,-0.04981269,0.03864716,0.144296063,0.008497743,0.07990766,0.12553220,0.125116874,0.04512335,⋯,0.473135015,8.172993e-02,2.472715e-04,0.2401524,0.3733626,0.66820690,0.1742185378,0.09551741,0.68155758,0.65590408
OGN,brown,-0.11131053,-0.12110708,-0.038577741,-0.016598778,0.01591847,-0.17446610,0.042950302,0.10417841,⋯,0.096926089,2.595904e-51,2.144848e-09,0.9133175,0.7752202,0.28179256,0.0112051557,0.26462611,0.18047328,0.25841961


In [52]:
mes %>%
    as_tibble(rownames = "sample_name") %>%
    dplyr::rename_if(startsWith(colnames(.), "ME"), ~ gsub("ME", "", .))

sample_name,blue,green,purple,magenta,pink,red,black,yellow,brown,turquoise,grey
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TCGA-C5-A1BF-01B-11R-A13Y-07,-0.003256702,0.073537564,0.0341830711,-0.0131508427,0.010626381,0.076211183,0.0236326101,0.005591921,0.030573462,0.129155183,-0.0408391432
TCGA-MU-A51Y-01A-11R-A26T-07,0.043969035,-0.047504170,-0.0314698674,-0.0218445354,-0.023973306,-0.005620746,-0.0060860641,0.082991029,0.110799549,0.024041363,-0.0026598050
TCGA-EK-A2RM-01A-21R-A18M-07,-0.032155831,0.044616210,-0.0307913185,-0.0129065499,-0.020022744,0.086350881,0.0240009505,-0.022432207,0.005870983,0.096964644,-0.0574064510
TCGA-Q1-A73P-01A-11R-A32P-07,-0.087552967,-0.067152253,-0.0049366333,0.0495055955,0.187615605,0.119583810,-0.0051471091,0.013082427,0.020945195,-0.025477452,-0.0517716770
TCGA-C5-A8YT-01A-11R-A37O-07,-0.129403614,-0.047925655,-0.0007305383,0.0470770746,0.034027389,-0.057191386,0.0392001546,-0.031724194,0.182165412,0.147411200,-0.0393792670
TCGA-C5-A2LZ-01A-11R-A213-07,0.058003190,-0.030624825,-0.0323186637,-0.0173489106,-0.025172427,-0.031825678,-0.0215443279,0.013620821,-0.045479206,-0.093695541,0.0617758609
TCGA-UC-A7PI-01A-11R-A42S-07,-0.107236168,-0.095558051,-0.0162119820,0.0665165490,0.146895877,0.083641949,-0.0196236258,-0.064048599,-0.028465621,-0.059999392,-0.0526778622
TCGA-VS-A9V1-01A-11R-A42T-07,-0.096191591,-0.067100951,0.0285122602,-0.0016588757,0.192821129,0.166728195,-0.0100744069,-0.050537649,-0.046350271,-0.057565138,-0.0575903659
TCGA-FU-A5XV-01A-11R-A28H-07,0.103065210,0.060812402,-0.0274597177,-0.0274295933,-0.028479613,0.025468069,-0.0207390534,-0.020395758,-0.029940526,-0.032622149,-0.0570406794


In [224]:
pivot_mm_gs_df <- joined_gs_mm_df %>%
    dplyr::select(-one_of(colnames(gene_gs_pval_df)[-1])) %>%
    pivot_longer(cols = colnames(gene_mm_df)[-1], names_to = "module_cor", values_to = "module_cor_val") %>%
    pivot_longer(cols = colnames(gene_gs_df)[-1], names_to = "trait_cor", values_to = "trait_cor_val") %>%
    pivot_longer(cols = colnames(gene_mm_pval_df)[-1], names_to = "module_pval", values_to = "module_pval_val")

In [226]:
head(pivot_mm_gs_df %>% dplyr::filter(trait_cor == "vital_status_cor", startsWith(module_cor, module), startsWith(module_pval, module)))

geneID,module,module_cor,module_cor_val,trait_cor,trait_cor_val,module_pval,module_pval_val
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>
PGF,blue,blue_cor,0.3840836,vital_status_cor,0.08506297,blue_pval,2.176317e-10
TIMP4,turquoise,turquoise_cor,0.2328224,vital_status_cor,0.09854756,turquoise_pval,1.759269e-04
C1QTNF6,turquoise,turquoise_cor,0.4255167,vital_status_cor,0.07354498,turquoise_pval,1.229518e-12
TNC,turquoise,turquoise_cor,0.5155294,vital_status_cor,0.07074473,turquoise_pval,1.025253e-18
PRL,purple,purple_cor,0.1442961,vital_status_cor,0.05597731,purple_pval,2.116778e-02
OGN,brown,brown_cor,0.7701209,vital_status_cor,-0.01796889,brown_pval,2.595904e-51


In [322]:
facet_plot <- function(pivot_df, trait_str, module_pval_thresh = 0.05, module_cor_pval_thresh = 0.5) {
    plt <- ggplot(pivot_df %>% dplyr::filter(trait_cor == paste0(trait_str, "_cor"), startsWith(module_cor, module), startsWith(module_pval, module))) +
        geom_point(aes(x = abs(module_cor_val), y = abs(trait_cor_val), color = (module_pval_val < module_pval_thresh) & (abs(module_cor_val) > module_cor_pval_thresh))) +
        geom_smooth(aes(x = abs(module_cor_val), y = abs(trait_cor_val)), method = "lm", color = "black") +
        facet_wrap(~ module, ncol = 3) +
        theme_classic() +
        labs(color = "Sig. in module", x = "gene abs(ME cor.)", y = paste0("gene abs(", gsub("_", " ", trait_str), " cor.)"))
    return(plt)
}

In [323]:
trait_strs <- c("vital_status", "figo_stage_1", "figo_stage_2", "figo_stage_3", "figo_stage_4")
for (ts in trait_strs) {
    facet_plot(pivot_mm_gs_df, ts)
    ggsave(filename = paste0(dirs$figures_dir, "/", unified_dsets[dset_idx], "_module_", ts, "_facet.png"), height = 10, width = 10)
}


`geom_smooth()` using formula 'y ~ x'
`geom_smooth()` using formula 'y ~ x'
`geom_smooth()` using formula 'y ~ x'
`geom_smooth()` using formula 'y ~ x'
`geom_smooth()` using formula 'y ~ x'
